# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from pprint import pprint
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
base_df = pd.read_csv('../WeatherPy/city_weather_data.csv',index_col=0)
base_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Qaanaaq,98,GL,1603494751,92,77.48,-69.36,18.82,7.92
1,Bredasdorp,31,ZA,1603494700,93,-34.53,20.04,66.20,0.87
2,Morehead,90,US,1603494889,88,37.27,-87.18,66.20,10.29
3,Santa Rosa,40,PH,1603494890,94,14.31,121.11,79.00,2.24
4,Hobyo,100,SO,1603494890,73,5.35,48.53,82.24,20.07


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
base_df = base_df.dropna()
locations = base_df[["Lat", "Lng"]].astype(float)
humidity = base_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = base_df.loc[base_df['Cloudiness'] < 10]
ideal_df = ideal_df.loc[ideal_df['Humidity'] < 40]
ideal_df = ideal_df.loc[ideal_df['Max Temp'] < 85]
ideal_df = ideal_df.loc[ideal_df['Max Temp'] > 50]
ideal_df = ideal_df.loc[ideal_df['Wind Speed'] < 10]

In [7]:
ideal_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
57,Rāmhormoz,0,IR,1603494905,38,31.28,49.60,65.5,3.02
126,Hwange,0,ZW,1603494932,30,-18.37,26.50,74.8,4.65
174,Tataouine,0,TN,1603494948,31,32.93,10.45,75.2,2.24
214,Saint George,1,US,1603494837,16,37.10,-113.58,77.0,5.57
233,Buraidah,0,SA,1603494966,32,26.33,43.98,66.2,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(inplace = True, drop = True)
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Rāmhormoz,0,IR,1603494905,38,31.28,49.60,65.5,3.02,
1,Hwange,0,ZW,1603494932,30,-18.37,26.50,74.8,4.65,
2,Tataouine,0,TN,1603494948,31,32.93,10.45,75.2,2.24,
3,Saint George,1,US,1603494837,16,37.10,-113.58,77.0,5.57,
4,Buraidah,0,SA,1603494966,32,26.33,43.98,66.2,4.70,


In [9]:
target_coordinates = ","

params = {"location": target_coordinates,
    "radius": 5000,
    "type": 'lodging',
    "key": g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [15]:
params['location']= f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
response = requests.get(base_url,params=params).json()
pprint(response['results'][0]['name'])

'فروشگاه مطورعلیان'


In [17]:
pprint(response)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 31.2779214, 'lng': 49.5990096},
                           'viewport': {'northeast': {'lat': 31.2792766802915,
                                                      'lng': 49.6003642802915},
                                        'southwest': {'lat': 31.2765787197085,
                                                      'lng': 49.59766631970851}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'فروشگاه مطورعلیان',
              'place_id': 'ChIJCUESjWIzxz8Rw6bQo5XNJQs',
              'plus_code': {'compound_code': '7HHX+5J Ramhormoz, Khuzestan '
                                             'Province, Iran',
                            'global_code': '8H3F7HHX+5J'},
              'reference': 'ChIJCUESjWIzxz8Rw6bQo5XNJQs',
              'scope': 'GOOGLE',
              'types': ['lodging', '

In [18]:
hotels = []
for x in range(len(hotel_df["City"])):
    params['location']= f"{hotel_df['Lat'][x]},{hotel_df['Lng'][x]}"
    #response = requests.get(base_url,params=params).json()
    hotels.append(response['results'][0]['name'])
hotel_df["Hotel Name"] = hotels

In [19]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Rāmhormoz,0,IR,1603494905,38,31.28,49.60,65.5,3.02,فروشگاه مطورعلیان
1,Hwange,0,ZW,1603494932,30,-18.37,26.50,74.8,4.65,فروشگاه مطورعلیان
2,Tataouine,0,TN,1603494948,31,32.93,10.45,75.2,2.24,فروشگاه مطورعلیان
3,Saint George,1,US,1603494837,16,37.10,-113.58,77.0,5.57,فروشگاه مطورعلیان
4,Buraidah,0,SA,1603494966,32,26.33,43.98,66.2,4.70,فروشگاه مطورعلیان


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))